In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
import emojis

In [2]:
from pyspark.sql import SparkSession

# Инициализируем SparkSession
spark = SparkSession.builder \
    .master('local') \
    .appName('Dreamlone') \
    .config('spark.executor.memory', '4gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

sc = spark.sparkContext

# Используем SQLContext чтобы прочитать файл
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Task 3

### get reposts of the original posts of the itmo group (posts.json) from user posts (the result should be similar to (group_post_id, Array (user_post_ids)))

In [3]:
itmo_posts = sqlContext.read.parquet('./bigdata20/posts_api.parquet')
fol_posts = spark.read.parquet("./bigdata20/followers_posts_api_final.parquet")


target = fol_posts.filter(fol_posts.copy_history.owner_id.getItem(0) == -94).select("key", "copy_history.id", "copy_history.owner_id") \
.withColumn("owner_id", element_at('owner_id',1)) \
.withColumn("id", element_at('id',1)) \
.select("owner_id","key","id") \
.selectExpr("cast(owner_id as string) owner_id", "cast(id as string) id", "key")
target.createOrReplaceTempView("df")
target = spark.sql("SELECT CONCAT(owner_id, '_',  id), key FROM df").toDF("itmo_posts","users_posts")

target.groupBy("itmo_posts").agg(collect_set("users_posts")).toDF("itmo_posts","posts")
target.coalesce(1).write.json("./bigdata20/Answers/task3_itmo_reposts.json")